In [ ]:
# نصب پکیج‌ها در Colab
!pip cache purge
!pip install flask==2.0.1 flask-socketio==5.3.6 eventlet==0.33.3
!pip install opencv-python-headless ultralytics gtts pyngrok requests
!pip install werkzeug==2.0.3 --force-reinstall  # downgrade werkzeug برای حل ImportError قبلی
!apt-get update && apt-get install -y libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

import cv2
import numpy as np
from ultralytics import YOLOWorld
from gtts import gTTS  # نگه داشتم اما پخش رو به کلاینت منتقل کردم
import os
from datetime import datetime
import threading
import queue
from flask import Flask, Response, render_template_string
from flask_socketio import SocketIO, emit
import time
from pyngrok import ngrok
import requests
import base64
import io
import atexit  # اضافه شده برای release خودکار writer

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*", async_mode='eventlet', max_http_buffer_size=10**7)  # افزایش بافر برای حل Too many packets

# تنظیمات
MODEL_PATH = "yolov8s-worldv2.pt"
MODEL_URL = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8s-worldv2.pt"
DISTANCE_THRESHOLD = 2.0
OUTPUT_DIR = "output"
VIDEO_FPS = 15
OBSTACLE_CLASSES = [
    'wall', 'pillar', 'door', 'stairs', 'chair', 'table', 'furniture', 'person', 'car',
    'tree', 'bench', 'trash can', 'bicycle', 'path', 'grass',
    'motorcycle', 'bus', 'traffic light', 'sign', 'sidewalk', 'pole', 'fence', 'ramp'
]
frame_queue = queue.Queue(maxsize=5)
detected_obstacles = []
alert_lock = threading.Lock()
last_alert_time = 0
ALERT_COOLDOWN = 2
VIDEO_OUT_PATH = os.path.join(OUTPUT_DIR, f"output_video_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4")
FRAME_SIZE = (320, 240)
video_writer = None

# تابع برای release writer در پایان اپ
def release_video_writer():
    global video_writer
    if video_writer is not None:
        video_writer.release()
        video_writer = None
        print(f"Video saved and released: {VIDEO_OUT_PATH}")

atexit.register(release_video_writer)  # ثبت برای release خودکار

# HTML و JavaScript به صورت رشته (با استایل بهبودیافته و مدیریت بهتر دوربین برای موبایل)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>هشدار موانع</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.7.5/socket.io.js"></script>
    <style>
        body {
            font-family: 'Tahoma', sans-serif; /* فونت مناسب برای فارسی */
            text-align: center;
            background-color: #f4f4f4; /* پس‌زمینه روشن */
            color: #333;
            margin: 0;
            padding: 20px;
        }
        h1 {
            color: #007bff; /* آبی جذاب */
            font-size: 2em;
            margin-bottom: 20px;
        }
        #webcam, #processed {
            width: 100%;
            max-width: 640px;
            display: block;
            margin: 10px auto;
            border: 2px solid #007bff; /* border آبی */
            border-radius: 8px; /* گوشه‌های گرد */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* سایه نرم */
        }
        .buttons {
            display: flex;
            justify-content: center;
            gap: 10px;
            flex-wrap: wrap;
        }
        button {
            padding: 10px 20px;
            background-color: #007bff; /* آبی دکمه‌ها */
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1em;
            transition: background-color 0.3s;
        }
        button:hover {
            background-color: #0056b3; /* تیره‌تر هنگام hover */
        }
        #canvas {
            display: none;
        }
        p {
            font-size: 1.1em;
            color: #555;
        }
        #status {
            color: #28a745; /* سبز برای وضعیت */
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>هشدار موانع</h1>
    <video id="webcam" autoplay playsinline width="320" height="240"></video>
    <canvas id="canvas" width="320" height="240"></canvas>
    <img id="processed" src="" alt="Processed Video">  <!-- اضافه شده: src خالی برای جلوگیری از خطا -->
    <div class="buttons">
        <button onclick="startVideo()">شروع</button>
        <button onclick="toggleCamera()">تعویض دوربین</button>  <!-- اضافه شده: دکمه تعویض دوربین -->
        <button onclick="stopVideo()">توقف</button>
    </div>
    <p>وضعیت: <span id="status">در انتظار شروع</span></p>

    <script>
        const socket = io();
        let video = document.getElementById('webcam');
        let canvas = document.getElementById('canvas');
        let context = canvas.getContext('2d');
        let intervalId;
        let currentStream = null;
        let currentFacingMode = 'user';  // شروع با دوربین جلو ('user' برای جلو، 'environment' برای عقب)
        const synth = window.speechSynthesis;
        let voices = [];

        // بارگذاری voices برای هشدار صوتی
        function loadVoices() {
            voices = synth.getVoices();
            if (voices.length === 0) {
                console.warn('No voices loaded yet.');
            } else {
                console.log('Voices loaded:', voices.map(v => v.lang + ' - ' + v.name));
            }
        }

        if ('onvoiceschanged' in synth) {
            synth.onvoiceschanged = loadVoices;
        }
        loadVoices();  // بارگذاری اولیه

        async function getCameraStream(facingMode) {
            const constraints = {
                video: {
                    width: { ideal: 320 },
                    height: { ideal: 240 },
                    facingMode: facingMode
                }
            };
            try {
                const stream = await navigator.mediaDevices.getUserMedia(constraints);
                return stream;
            } catch (err) {
                console.error('Error accessing camera:', err.name + ' - ' + err.message);
                document.getElementById('status').innerText = 'خطا در دسترسی به دوربین: ' + err.name + ' - ' + err.message + ' (مجوزها را چک کنید یا دستگاه را ریستارت کنید)';
                throw err;
            }
        }

        function stopCurrentStream() {
            if (currentStream) {
                currentStream.getTracks().forEach(track => track.stop());
                currentStream = null;
            }
        }

        async function startVideo() {
            stopCurrentStream();
            try {
                currentStream = await getCameraStream(currentFacingMode);
                video.srcObject = currentStream;
                intervalId = setInterval(captureFrame, 150);
                document.getElementById('status').innerText = 'شروع شد';
            } catch (err) {
                // handled in getCameraStream
            }
        }

        async function toggleCamera() {
            if (!currentStream) {
                console.warn('No stream to toggle');
                return;
            }
            stopCurrentStream();
            currentFacingMode = currentFacingMode === 'user' ? 'environment' : 'user';
            try {
                currentStream = await getCameraStream(currentFacingMode);
                video.srcObject = currentStream;
            } catch (err) {
                // handled in getCameraStream
            }
        }

        function stopVideo() {
            if (intervalId) clearInterval(intervalId);
            stopCurrentStream();
            document.getElementById('status').innerText = 'توقف شد';
            socket.emit('stop_video');  // ارسال برای release ویدیو
        }

        function captureFrame() {
            context.drawImage(video, 0, 0, 320, 240);
            let dataUrl = canvas.toDataURL('image/jpeg', 0.5);  // کیفیت پایین‌تر برای موبایل (برای سرعت بیشتر)
            socket.emit('image', dataUrl);
        }

        socket.on('processed_image', function(data) {
            document.getElementById('processed').src = 'data:image/jpeg;base64,' + data;
        });

        socket.on('alert', function(text) {
            if (synth.speaking) {
                synth.cancel();  // اگر در حال پخش باشه، متوقف کن
            }
            let utterance = new SpeechSynthesisUtterance(text);
            utterance.lang = 'fa-IR';  // سعی برای فارسی
            // انتخاب voice مناسب اگر موجود باشه
            const preferredVoice = voices.find(v => v.lang.startsWith('fa') || v.lang.startsWith('en'));
            if (preferredVoice) {
                utterance.voice = preferredVoice;
            } else {
                utterance.lang = 'en-US';  // fallback به انگلیسی
            }
            utterance.onerror = function(event) {
                console.error('Speech error:', event.error);
            };
            synth.speak(utterance);
        });

        socket.on('video_saved', function(path) {
            console.log('Video saved at: ' + path);
            alert('ویدیو ذخیره شد: ' + path);  // اطلاع‌رسانی به کاربر
        });

        socket.on('connect_error', function(err) {
            console.error('Socket connection error:', err);
            document.getElementById('status').innerText = 'خطا در اتصال سرور';
        });
    </script>
</body>
</html>
"""

# ذخیره HTML در templates
os.makedirs('templates', exist_ok=True)
with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(HTML_TEMPLATE)

# ایجاد دایرکتوری خروجی
os.makedirs(OUTPUT_DIR, exist_ok=True)

# تابع برای دانلود مدل
def download_model(url, path):
    if not os.path.exists(path):
        print(f"Downloading model from {url}...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Model downloaded to {path}")
        else:
            raise Exception(f"Failed to download model from {url}")
    else:
        print(f"Model already exists at {path}")

# دانلود و لود مدل
download_model(MODEL_URL, MODEL_PATH)
model = YOLOWorld(MODEL_PATH)
model.set_classes(OBSTACLE_CLASSES)

def estimate_distance(box):
    box_width = box[2] - box[0]
    focal_length = 500
    avg_object_width = 0.5
    distance = (avg_object_width * focal_length) / box_width
    steps = round(float(distance / 0.7))
    return float(distance), steps

def generate_alert(text):
    global last_alert_time
    current_time = time.time()
    if current_time - last_alert_time < ALERT_COOLDOWN:
        return
    last_alert_time = current_time
    socketio.emit('alert', text, namespace='/')  # تغییر به socketio.emit با namespace explicit برای حل context error

def process_frame(frame):
    global video_writer
    if video_writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(VIDEO_OUT_PATH, fourcc, VIDEO_FPS, FRAME_SIZE)

    frame = cv2.resize(frame, FRAME_SIZE)
    results = model.predict(frame, conf=0.5, verbose=False)
    obstacles_in_frame = []
    annotated_frame = frame.copy()

    for result in results:
        for box in result.boxes:
            label = result.names[int(box.cls)]
            if label.lower() in [cls.lower() for cls in OBSTACLE_CLASSES]:
                confidence = float(box.conf)
                if confidence > 0.5:
                    obstacles_in_frame.append(label)
                    distance, steps = estimate_distance(box.xyxy[0])
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    if distance < DISTANCE_THRESHOLD:
                        with alert_lock:
                            alert_text = f"Warning: {label} at {steps} steps away from you"
                            socketio.start_background_task(generate_alert, alert_text)  # تغییر به start_background_task برای حل thread issues
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # تغییر: قرمز و ضخامت 2 برای وضوح بیشتر
                    cv2.putText(annotated_frame, f"{label} {steps} steps",
                                (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)  # بزرگ‌تر برای وضوح

    if video_writer is not None:
        video_writer.write(annotated_frame)

    return annotated_frame, obstacles_in_frame

@socketio.on('image')
def handle_image(data):
    try:
        img_data = base64.b64decode(data.split(',')[1])
        img_np = np.frombuffer(img_data, np.uint8)
        frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
        if frame is None:
            print("Error: Could not decode frame")
            return
        annotated_frame, obstacles = process_frame(frame)
        ret, buffer = cv2.imencode('.jpg', annotated_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 70])  # کیفیت 70% برای نمایش بهتر
        if not ret:
            print("Error: Could not encode frame")
            return
        annotated_base64 = base64.b64encode(buffer).decode('utf-8')
        emit('processed_image', annotated_base64)
    except Exception as e:
        print(f"Error processing image: {e}")

@socketio.on('stop_video')
def handle_stop_video():
    global video_writer
    if video_writer is not None:
        video_writer.release()
        video_writer = None
        emit('video_saved', VIDEO_OUT_PATH)
        print(f"Video released and saved: {VIDEO_OUT_PATH}")

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

ngrok.set_auth_token(os.getenv('NGROK_TOKEN', '2s9UYBDAhKCGblwUTV8PPf5AhJg_75PDRoniQ2hhzvtmoy8SN'))
def keep_alive():
    while True:
        time.sleep(3600)
threading.Thread(target=keep_alive, daemon=True).start()

if __name__ == '__main__':
    http_tunnel = ngrok.connect(5000, bind_tls=True)
    public_url = http_tunnel.public_url
    print(f"Flask app running at: {public_url}")
    socketio.run(app, port=5000, debug=False)

Files removed: 0
ERROR: Operation cancelled by user
'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.11/warnings.py", line 570, in <module>
    _processoptions(sys.warnoptions)
  File "/usr/lib/python3.11/warnings.py", line 209, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.11/warnings.py", line 225, in _setoption
    import re
  File "/usr/lib/python3.11/re/__init__.py", line 124, in <module>
    import enum
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1069, in get_code
  File "<frozen importlib._bootstrap_external>", line 729, in _compile_bytecode
KeyboardInterrupt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.6 MB

In [ ]:
# نصب پکیج‌ها در Colab
!pip cache purge
!pip install flask==2.0.1 flask-socketio==5.3.6 eventlet==0.33.3
!pip install opencv-python-headless ultralytics gtts pyngrok requests
!pip install werkzeug==2.0.3 --force-reinstall  # downgrade werkzeug برای حل ImportError قبلی
!apt-get update && apt-get install -y libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

import cv2
import numpy as np
from ultralytics import YOLOWorld
from gtts import gTTS  # نگه داشتم اما پخش رو به کلاینت منتقل کردم
import os
from datetime import datetime
import threading
import queue
from flask import Flask, Response, render_template_string
from flask_socketio import SocketIO, emit
import time
from pyngrok import ngrok
import requests
import base64
import io
import atexit  # اضافه شده برای release خودکار writer

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*", async_mode='eventlet', max_http_buffer_size=10**7)  # افزایش بافر برای حل Too many packets

# تنظیمات
MODEL_PATH = "yolov8x-worldv2.pt"  # تغییر: مدل دقیق‌تر (extra-large)
MODEL_URL = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8x-worldv2.pt"  # تغییر: URL مدل جدید
DISTANCE_THRESHOLD = 2.0
OUTPUT_DIR = "output"
VIDEO_FPS = 15
OBSTACLE_CLASSES = [
    'stop sign', 'person', 'bicycle', 'bus', 'truck', 'car', 'motorbike', 'reflective cone',
    'ashcan', 'warning column', 'spherical roadblock', 'pole', 'dog', 'tricycle', 'fire hydrant',
    'tree', 'rock', 'branch', 'root', 'boulder', 'cliff', 'stream', 'animal', 'stairs', 'staircase', 'steps',  # تغییر: کلاس‌های بیشتر برای پله
    'escalator', 'elevator', 'door', 'bench', 'platform edge', 'crowd', 'sign',
    'chair', 'wooden chair', 'plastic chair', 'metal chair', 'table', 'coffee table', 'furniture', 'rug', 'cord', 'wire', 'toy',
    'knife', 'kitchen knife', 'stove', 'oven', 'sharp object', 'glass', 'broken glass', 'window', 'pet', 'cable', 'lamp', 'vase', 'book', 'shelf',
    'house door', 'wooden door', 'column', 'street light pole', 'utility pole', 'electricity pole', 'lamp post'
]
DANGEROUS_CLASSES = [
    'stairs', 'staircase', 'steps', 'knife', 'kitchen knife', 'stove', 'oven', 'sharp object', 'glass', 'broken glass', 'cliff', 'stream', 'rock',  # تغییر: کلاس‌های بیشتر برای پله
    'root', 'boulder', 'escalator', 'platform edge', 'fire hydrant', 'pole', 'cord', 'wire', 'cable', 'column', 'street light pole', 'utility pole', 'electricity pole', 'lamp post'
]
frame_queue = queue.Queue(maxsize=5)
detected_obstacles = []
alert_lock = threading.Lock()
last_alert_time = 0
ALERT_COOLDOWN = 2
VIDEO_OUT_PATH = os.path.join(OUTPUT_DIR, f"output_video_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4")
FRAME_SIZE = (640, 480)  # تغییر: افزایش رزولوشن برای دقت بالاتر در تشخیص اشیاء کوچک
video_writer = None

# تابع برای release writer در پایان اپ
def release_video_writer():
    global video_writer
    if video_writer is not None:
        video_writer.release()
        video_writer = None
        print(f"Video saved and released: {VIDEO_OUT_PATH}")

atexit.register(release_video_writer)  # ثبت برای release خودکار

# HTML و JavaScript به صورت رشته (با استایل بهبودیافته و مدیریت بهتر دوربین برای موبایل)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>هشدار موانع</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.7.5/socket.io.js"></script>
    <style>
        body {
            font-family: 'Tahoma', sans-serif; /* فونت مناسب برای فارسی */
            text-align: center;
            background-color: #f4f4f4; /* پس‌زمینه روشن */
            color: #333;
            margin: 0;
            padding: 20px;
        }
        h1 {
            color: #007bff; /* آبی جذاب */
            font-size: 2em;
            margin-bottom: 20px;
        }
        #webcam, #processed {
            width: 100%;
            max-width: 640px;
            display: block;
            margin: 10px auto;
            border: 2px solid #007bff; /* border آبی */
            border-radius: 8px; /* گوشه‌های گرد */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* سایه نرم */
        }
        .buttons {
            display: flex;
            justify-content: center;
            gap: 10px;
            flex-wrap: wrap;
        }
        button {
            padding: 10px 20px;
            background-color: #007bff; /* آبی دکمه‌ها */
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1em;
            transition: background-color 0.3s;
        }
        button:hover {
            background-color: #0056b3; /* تیره‌تر هنگام hover */
        }
        #canvas {
            display: none;
        }
        p {
            font-size: 1.1em;
            color: #555;
        }
        #status {
            color: #28a745; /* سبز برای وضعیت */
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>هشدار موانع</h1>
    <video id="webcam" autoplay playsinline width="640" height="480"></video>  <!-- تغییر: تطبیق با رزولوشن جدید -->
    <canvas id="canvas" width="640" height="480"></canvas>  <!-- تغییر: تطبیق با رزولوشن جدید -->
    <img id="processed" src="" alt="Processed Video">  <!-- اضافه شده: src خالی برای جلوگیری از خطا -->
    <div class="buttons">
        <button onclick="startVideo()">شروع</button>
        <button onclick="toggleCamera()">تعویض دوربین</button>  <!-- اضافه شده: دکمه تعویض دوربین -->
        <button onclick="stopVideo()">توقف</button>
    </div>
    <p>وضعیت: <span id="status">در انتظار شروع</span></p>

    <script>
        const socket = io();
        let video = document.getElementById('webcam');
        let canvas = document.getElementById('canvas');
        let context = canvas.getContext('2d');
        let intervalId;
        let currentStream = null;
        let currentFacingMode = 'user';  // شروع با دوربین جلو ('user' برای جلو، 'environment' برای عقب)
        const synth = window.speechSynthesis;
        let voices = [];

        // بارگذاری voices برای هشدار صوتی
        function loadVoices() {
            voices = synth.getVoices();
            if (voices.length === 0) {
                console.warn('No voices loaded yet.');
            } else {
                console.log('Voices loaded:', voices.map(v => v.lang + ' - ' + v.name));
            }
        }

        if ('onvoiceschanged' in synth) {
            synth.onvoiceschanged = loadVoices;
        }
        loadVoices();  // بارگذاری اولیه

        async function getCameraStream(facingMode) {
            const constraints = {
                video: {
                    width: { ideal: 640 },  // تغییر: تطبیق با رزولوشن جدید
                    height: { ideal: 480 },  // تغییر: تطبیق با رزولوشن جدید
                    facingMode: facingMode
                }
            };
            try {
                const stream = await navigator.mediaDevices.getUserMedia(constraints);
                return stream;
            } catch (err) {
                console.error('Error accessing camera:', err.name + ' - ' + err.message);
                document.getElementById('status').innerText = 'خطا در دسترسی به دوربین: ' + err.name + ' - ' + err.message + ' (مجوزها را چک کنید یا دستگاه را ریستارت کنید)';
                throw err;
            }
        }

        function stopCurrentStream() {
            if (currentStream) {
                currentStream.getTracks().forEach(track => track.stop());
                currentStream = null;
            }
        }

        async function startVideo() {
            stopCurrentStream();
            try {
                currentStream = await getCameraStream(currentFacingMode);
                video.srcObject = currentStream;
                intervalId = setInterval(captureFrame, 150);
                document.getElementById('status').innerText = 'شروع شد';
            } catch (err) {
                // handled in getCameraStream
            }
        }

        async function toggleCamera() {
            if (!currentStream) {
                console.warn('No stream to toggle');
                return;
            }
            stopCurrentStream();
            currentFacingMode = currentFacingMode === 'user' ? 'environment' : 'user';
            try {
                currentStream = await getCameraStream(currentFacingMode);
                video.srcObject = currentStream;
            } catch (err) {
                // handled in getCameraStream
            }
        }

        function stopVideo() {
            if (intervalId) clearInterval(intervalId);
            stopCurrentStream();
            document.getElementById('status').innerText = 'توقف شد';
            socket.emit('stop_video');  // ارسال برای release ویدیو
        }

        function captureFrame() {
            context.drawImage(video, 0, 0, 640, 480);  // تغییر: تطبیق با رزولوشن جدید
            let dataUrl = canvas.toDataURL('image/jpeg', 0.5);  // کیفیت پایین‌تر برای موبایل (برای سرعت بیشتر)
            socket.emit('image', dataUrl);
        }

        socket.on('processed_image', function(data) {
            document.getElementById('processed').src = 'data:image/jpeg;base64,' + data;
        });

        socket.on('alert', function(data) {
            if (synth.speaking) {
                synth.cancel();  // اگر در حال پخش باشه، متوقف کن
            }
            let utterance = new SpeechSynthesisUtterance(data.text);
            utterance.lang = 'fa-IR';  // سعی برای فارسی
            // انتخاب voice مناسب اگر موجود باشه
            const preferredVoice = voices.find(v => v.lang.startsWith('fa') || v.lang.startsWith('en'));
            if (preferredVoice) {
                utterance.voice = preferredVoice;
            } else {
                utterance.lang = 'en-US';  // fallback به انگلیسی
            }
            utterance.volume = data.is_dangerous ? 1.0 : 0.5;  // حجم کمتر برای غیرخطرناک
            utterance.onerror = function(event) {
                console.error('Speech error:', event.error);
            };
            synth.speak(utterance);
            if (data.is_dangerous && 'vibrate' in navigator) {
                navigator.vibrate(100);  // لرزش ریز (100 میلی‌ثانیه) برای اشیاء خطرناک
            }
        });

        socket.on('video_saved', function(path) {
            console.log('Video saved at: ' + path);
            alert('ویدیو ذخیره شد: ' + path);  // اطلاع‌رسانی به کاربر
        });

        socket.on('connect_error', function(err) {
            console.error('Socket connection error:', err);
            document.getElementById('status').innerText = 'خطا در اتصال سرور';
        });
    </script>
</body>
</html>
"""

# ذخیره HTML در templates
os.makedirs('templates', exist_ok=True)
with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(HTML_TEMPLATE)

# ایجاد دایرکتوری خروجی
os.makedirs(OUTPUT_DIR, exist_ok=True)

# تابع برای دانلود مدل
def download_model(url, path):
    if not os.path.exists(path):
        print(f"Downloading model from {url}...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Model downloaded to {path}")
        else:
            raise Exception(f"Failed to download model from {url}")
    else:
        print(f"Model already exists at {path}")

# دانلود و لود مدل
download_model(MODEL_URL, MODEL_PATH)
model = YOLOWorld(MODEL_PATH)
model.set_classes(OBSTACLE_CLASSES)

def estimate_distance(box):
    box_width = box[2] - box[0]
    focal_length = 500
    avg_object_width = 0.5
    distance = (avg_object_width * focal_length) / box_width
    steps = round(float(distance / 0.7))
    return float(distance), steps

def generate_alert(text, is_dangerous):
    global last_alert_time
    current_time = time.time()
    if current_time - last_alert_time < ALERT_COOLDOWN:
        return
    last_alert_time = current_time
    socketio.emit('alert', {'text': text, 'is_dangerous': is_dangerous}, namespace='/')

def process_frame(frame):
    global video_writer
    if video_writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(VIDEO_OUT_PATH, fourcc, VIDEO_FPS, FRAME_SIZE)

    frame = cv2.resize(frame, FRAME_SIZE)
    results = model.predict(frame, conf=0.7, iou=0.45, imgsz=640, verbose=False)  # تغییر: تنظیمات برای دقت بالاتر در اشیاء کوچک
    obstacles_in_frame = []
    annotated_frame = frame.copy()

    width = FRAME_SIZE[0]

    for result in results:
        for box in result.boxes:
            label = result.names[int(box.cls)]
            if label.lower() in [cls.lower() for cls in OBSTACLE_CLASSES]:
                confidence = float(box.conf)
                if confidence > 0.7:  # تغییر: همگام با conf جدید
                    obstacles_in_frame.append(label)
                    distance, steps = estimate_distance(box.xyxy[0])
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    center_x = (x1 + x2) / 2
                    if center_x < width / 3:
                        direction = "on the left"
                    elif center_x > 2 * width / 3:
                        direction = "on the right"
                    else:
                        direction = "in the center"
                    is_dangerous = label.lower() in [cls.lower() for cls in DANGEROUS_CLASSES]
                    alert_prefix = "Danger: " if is_dangerous else "Notice: "
                    if distance < DISTANCE_THRESHOLD:
                        with alert_lock:
                            alert_text = f"{alert_prefix}{label} {direction} at {steps} steps away from you"
                            socketio.start_background_task(generate_alert, alert_text, is_dangerous)
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # تغییر: قرمز و ضخامت 2 برای وضوح بیشتر
                    cv2.putText(annotated_frame, f"{alert_prefix}{label} {direction} {steps} steps",
                                (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)  # بزرگ‌تر برای وضوح

    if video_writer is not None:
        video_writer.write(annotated_frame)

    return annotated_frame, obstacles_in_frame

@socketio.on('image')
def handle_image(data):
    try:
        img_data = base64.b64decode(data.split(',')[1])
        img_np = np.frombuffer(img_data, np.uint8)
        frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
        if frame is None:
            print("Error: Could not decode frame")
            return
        annotated_frame, obstacles = process_frame(frame)
        ret, buffer = cv2.imencode('.jpg', annotated_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 70])  # کیفیت 70% برای نمایش بهتر
        if not ret:
            print("Error: Could not encode frame")
            return
        annotated_base64 = base64.b64encode(buffer).decode('utf-8')
        emit('processed_image', annotated_base64)
    except Exception as e:
        print(f"Error processing image: {e}")

@socketio.on('stop_video')
def handle_stop_video():
    global video_writer
    if video_writer is not None:
        video_writer.release()
        video_writer = None
        emit('video_saved', VIDEO_OUT_PATH)
        print(f"Video released and saved: {VIDEO_OUT_PATH}")

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

ngrok.set_auth_token(os.getenv('NGROK_TOKEN', '2s9UYBDAhKCGblwUTV8PPf5AhJg_75PDRoniQ2hhzvtmoy8SN'))
def keep_alive():
    while True:
        time.sleep(3600)
threading.Thread(target=keep_alive, daemon=True).start()

if __name__ == '__main__':
    http_tunnel = ngrok.connect(5000, bind_tls=True)
    public_url = http_tunnel.public_url
    print(f"Flask app running at: {public_url}")
    socketio.run(app, port=5000, debug=False)

Files removed: 6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W

In [ ]:
import os
os.kill(os.getpid(), 9)